* 本文结合EDA进行规则设计,初赛可以认为是短期时间序列预测,所以越近的数据相关性越强,但很多时候我们又不可避免的遇到奇异值的情况，而奇异值的出现又会大大影响我们的预测, 本篇文章我们采用上周的分钟&小时均值来减小奇异值的影响，针对该问题设计了一种规则, 该规则可以达到LB 12.10, 即初赛top2%的成绩。

1  工具包导入&数据读取
    * 1.1  工具包导入
    * 1.2  数据读取
2  EDA
    * 2.1  数据准备
        * 2.1.1  将数据转化为10分钟的单位
        * 2.1.2  获取inNums&outNums
    * 2.2  最新一周每条地铁线观测
        * 2.2.1  0号站点观察
        * 2.2.2  1号站点观察
    * 2.3  查看可能带来较大误差的地铁线
        * 2.3.1  15号站点观察
            * 2.3.1.1  Innums
            * 2.3.1.2  Outnums
        * 2.3.2  9号站点观察
            * 2.3.2.1  Innums
            * 2.3.2.2  Outnums
3  规则建模
    * 3.1  统计10分钟均值特征
    * 3.2  统计小时均值特征
    * 3.3  计算比例
        * 3.3.1  压缩
    * 3.4  计算28日小时的均值
    * 3.5  预测： 29日的10min中预测结果 = 上周10min中的值 / 对应的小时均值 * 28日的小时均值
4  小结

### 1  工具包导入&数据读取

#### 1  工具包导入&数据读取

In [2]:
## 数据工具包
import numpy as np
np.random.seed(42)
import pandas as pd
from tqdm import tqdm
from tqdm import tqdm_notebook

import warnings
warnings.filterwarnings('ignore')

import xgboost as xgb
import lightgbm as lgb
from functools import partial

import os 
import gc
from scipy.sparse import vstack  
import time
import datetime

import multiprocessing as mp
import seaborn as sns 
%matplotlib inline

#### 1.2  数据读取

In [ ]:
path ='/data/TC_Underground/'
train = pd.DataFrame()
for i in tqdm(range(1,26)):
    if i < 10:
        train_tmp = pd.read_csv(path + 'record_2019-01-0' + str(i) + '.csv')
    else:
        train_tmp = pd.read_csv(path + 'record_2019-01-' + str(i) + '.csv')
    if i== 1:
        train = train_tmp
    else:
        train = pd.concat([train, train_tmp],axis=0,ignore_index=True) 

test_A_record  = pd.read_csv(path + 'testA_record_2019-01-28.csv')  
train  = pd.concat([train, test_A_record],axis=0, ignore_index = True)
train.head()

### 2  EDA

* 好的规则的设计都是基于好的EDA,如果没有很好的EDA,规则是较难设计的,所以此处我们先做EDA,那么在时序问题的时候最简单的EDA该从何做起呢？因为本次的赛题天池只给了20多天的数据,其中还有一些特殊的时间,例如1.1日是元旦期间,那个时候的分布很明显会和其他时间段不一样,不过幸运的是我们只需要预测29日的地铁10分钟流量。

#### 2.1  数据准备

##### 2.1.1  将数据转化为10分钟的单位
* 此处和需要提交的sub达到类似的形式(以10min为单位)

In [ ]:
def trans_time_10_minutes(x):
    x_split = x.split(':')
    x_part1 = x_split[0]
    x_part2 = int(x_split[1]) // 10
    if x_part2 == 0:
        x_part2 = '00'
    else:
        x_part2 = str(x_part2 * 10)
    return x_part1 + ':' + x_part2 + ':00'
     
train['startTime'] = train['time'].astype(str).apply(lambda x: trans_time_10_minutes(x))

* 时间信息拆解

In [ ]:
# train就是1-25号所有统计的记录，按照题目要求作出十分钟级别的dataframe。
# 通过画图，我们可以发现，1.21-1.25图形走势高度相似，同时和1.28的整体走势也较为相似。
train['days']    = pd.to_datetime(train['startTime'],format='%Y-%m-%d %H:%M:%S').dt.day
train['hours']   = pd.to_datetime(train['startTime'],format='%Y-%m-%d %H:%M:%S').dt.hour
train['minutes'] = pd.to_datetime(train['startTime'],format='%Y-%m-%d %H:%M:%S').dt.minute
train['wkday']   = pd.to_datetime(train['startTime'],format='%Y-%m-%d %H:%M:%S').dt.weekday 

#### 2.1.2  获取inNums&outNums

In [ ]:
train_10minutes = train.groupby(['stationID','startTime','wkday','days','hours','minutes'])['status'].sum().to_frame('inNums').reset_index()
train_10minutes.head()

In [ ]:
train_10minutes['outNums']  = train.groupby(['stationID','startTime','wkday','days','hours','minutes'])['status'].count().values
train_10minutes['outNums']  = train_10minutes['outNums']  - train_10minutes['inNums']


### 2.2  最新一周每条地铁线观测

* 因为有80多条路线,我们的规则设计主要是希望找到相近分布的曲线,而在时序问题中,越近的数据的关联性往往越大,所以我们选取最近的一周的数据进行观测.


#### 2.2.1  0号站点观察

* 从下面的图中,我们发现工作日的分布基本都是类似的.同时21日和28日的分布是相近的.(因而我们可以从上周的数据中进行规则的设计)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=[8,8])
plt.xlabel('Time')
plt.ylabel('Count')
for day in [21,22,23,24,25,28]:
    tmp = train_10minutes.loc[(train_10minutes.days == day) & (train_10minutes.stationID == 0)].copy()
    tmp = tmp.sort_values(['stationID','startTime'])
    plt.plot(tmp['startTime'],tmp['inNums'])

#### 2.2.2  1号站点观察

* 和上面的0号站点是一样的,
* 下面我们选取有代表的站点进行分析。

In [ ]:
plt.figure(figsize=[8,8])
plt.xlabel('Time')
plt.ylabel('Count')
for day in [21,22,23,24,25,28]:
    tmp = train_10minutes.loc[(train_10minutes.days == day) & (train_10minutes.stationID == 1)].copy()
    tmp = tmp.sort_values(['stationID','startTime'])
    plt.plot(tmp['startTime'],tmp['inNums'])

### 2.3  查看可能带来较大误差的地铁线

* 往往数值越大的出入也是带来误差最大的站点。
* 15,9,4,7,10号站点的流量较大,这些站点往往也是我们的误差所在

In [ ]:
train_10minutes.loc[train_10minutes.days.isin([21,22,23,24,25,28])].groupby(['stationID','days'])['inNums'].sum().sort_values(ascending = False)

In [ ]:
train_10minutes.loc[train_10minutes.days.isin([21,22,23,24,25,28])].groupby(['stationID','days'])['outNums'].sum().sort_values(ascending = False)

#### 2.3.1  15号站点观察

* 15号的站点的分布28和21有些许差别,峰值部分变小了,但是整体分布看上去还是相差不大

In [ ]:
plt.figure(figsize=[8,8])
plt.xlabel('Time')
plt.ylabel('Count')
for day in [21,22,23,24,25,28]:
    tmp = train_10minutes.loc[(train_10minutes.days == day) & (train_10minutes.stationID == 15)].copy()
    tmp = tmp.sort_values(['stationID','startTime'])
    plt.plot(tmp['startTime'],tmp['inNums'])

In [ ]:
plt.figure(figsize=[8,8])
plt.xlabel('Time')
plt.ylabel('Count')
for day in [21,22,23,24,25,28]:
    tmp = train_10minutes.loc[(train_10minutes.days == day) & (train_10minutes.stationID == 15)].copy()
    tmp = tmp.sort_values(['stationID','startTime'])
    plt.plot(tmp['startTime'],tmp['outNums'])

##### 2.3.1.1  Innums

* 我们单独把21日和28日的曲线拿出来进行观察,发现28日每个时间点的流量是下降的,也就是说29日可能也会下降.(这个时候据说是返程阶段,很多人已经回家了,所以地铁的流量可能会下降)

In [ ]:
tmp = train_10minutes.loc[(train_10minutes.days.isin([21,28]))]
tmp.loc[tmp.stationID == 15].pivot_table(index='hours',columns='days',values='inNums').plot(style='o-')

##### 2.3.1.2  Outnums

* 很奇怪的是,28日的出站好像很多都比21日的要高一些,但是总体分布还是类似的。

In [ ]:
tmp = train_10minutes.loc[(train_10minutes.days.isin([21,28]))]
tmp.loc[tmp.stationID == 15].pivot_table(index='hours',columns='days',values='outNums').plot(style='o-')

#### 2.3.2  9号站点观察

In [ ]:
plt.figure(figsize=[8,8])
plt.xlabel('Time')
plt.ylabel('Count')
for day in [21,22,23,24,25,28]:
    tmp = train_10minutes.loc[(train_10minutes.days == day) & (train_10minutes.stationID == 9)].copy()
    tmp = tmp.sort_values(['stationID','startTime'])
    plt.plot(tmp['startTime'],tmp['inNums'])

In [ ]:
plt.figure(figsize=[8,8])
plt.xlabel('Time')
plt.ylabel('Count')
for day in [21,22,23,24,25,28]:
    tmp = train_10minutes.loc[(train_10minutes.days == day) & (train_10minutes.stationID == 9)].copy()
    tmp = tmp.sort_values(['stationID','startTime'])
    plt.plot(tmp['startTime'],tmp['outNums'])

#### 2.3.2.1  Innums

In [ ]:
tmp = train_10minutes.loc[(train_10minutes.days.isin([21,28]))]
tmp.loc[tmp.stationID == 9].pivot_table(index='hours',columns='days',values='inNums').plot(style='o-') #title='销量',

##### 2.3.2.2  Outnums

In [ ]:
tmp = train_10minutes.loc[(train_10minutes.days.isin([21,28]))]
tmp.loc[tmp.stationID == 9].pivot_table(index='hours',columns='days',values='outNums').plot(style='o-') #title='销量',

### 3  规则建模
* 从上面的EDA中,我们发现相邻一周的每个站点的分布是类似的,于是我们便可以得到很多简单的规则建模,最容易想到的有:
* 基于相邻两天的分布是强相关的特征,我们可以用前一天的预测后一天的,参见林有夕的开源.
* 基于相邻一周中周一的分布观测情况,我们假设我们数据的分布存在周期分布,所以我们可以直接用上一周周二(22日)的信息预测29日的结果.
* 但是上面两个方法还是有些太简单了,此处我们采用一种平滑的方案.

#### 3.1  统计10分钟均值特征

统计上周每个十分钟段的in和out的均值


In [ ]:
# 第一步，我们先做成十分钟的均值/小时级均值，得到每十分钟相对于本小时的比例。因为相对而言，小时级的稳定性更高
t21_25 =train_10minutes[(train_10minutes['days']>=21)&(train_10minutes['days']<=25)]
t21_25_in = t21_25.groupby(['stationID', 'hours', 'minutes'])['inNums'].agg({'in_mean':'mean'}).reset_index()
t21_25_out =t21_25.groupby(['stationID', 'hours', 'minutes'])['outNums'].agg({'out_mean':'mean'}).reset_index()
t21_25_in_out = t21_25_in.merge(t21_25_out,on = ['stationID', 'hours', 'minutes'],how = 'left')
t21_25_in_out.head()

#### 3.2  统计小时均值特征

统计上周每小时时段的in和out的均值

In [ ]:
t21_25_in_hour = t21_25.groupby(['stationID', 'hours'])['inNums'].agg({'in_mean_hour':'mean'}).reset_index()
t21_25_out_hour = t21_25.groupby(['stationID', 'hours'])['outNums'].agg({'out_mean_hour':'mean'}).reset_index()
t21_25_in_out_hour = t21_25_in_hour.merge(t21_25_out_hour,on = ['stationID', 'hours'],how = 'left')
t21_25_in_out_hour.head()

#### 3.3  计算比例

计算每十分钟占该小时的比例

In [ ]:
t21_25_in_out = t21_25_in_out.merge(t21_25_in_out_hour,on = ['stationID', 'hours'],how = 'left')
t21_25_in_out['in_bili'] = t21_25_in_out['in_mean'] / (t21_25_in_out['in_mean_hour']+0.001)
t21_25_in_out['out_bili'] = t21_25_in_out['out_mean']/ (t21_25_in_out['out_mean_hour']+0.001)
t21_25_in_out_bili = t21_25_in_out[['stationID', 'hours', 'minutes','in_bili','out_bili']]

#### 3.3.1  压缩

* 有的时候可能会出现某一天突发情况,会出现某一时刻值特别大,那这个时候可能需要控制一下,此处我们对6点前这种情况进行压缩,如果某一天除以均值大于2,那么我们将其压缩至2.
* 凌晨时候，由于有地铁员工刷卡进站，其实人流量极少，在做比例的时候，可能会被突然放大，本身loss是mae，这种极小值没不要在乎，直接压缩他们的比例

In [ ]:
# 要加一个function判断，把小于6点的记录，大于2的都强制压倒2以内。我忘记咋写了，丢在家里了。因为之前除数里面有0.001，有可能会突然偏大。改好了！
def function(a, b):
    if a>=2 and b <=6: 
        return 2
    else:
        return a

t21_25_in_out_bili['in_bili']  = t21_25_in_out_bili.apply(lambda x: function(x.in_bili, x.hours), axis = 1)
t21_25_in_out_bili['out_bili'] = t21_25_in_out_bili.apply(lambda x: function(x.out_bili, x.hours), axis = 1)

### 3.4  计算28日小时的均值

In [ ]:
# 本规则的主要思想，就是利用相似性，预测目标的1.29，而时序问题是短期相似的，那么1.28和1.29的之间更为相似。
# 获取28号记录
t28 = train_10minutes[(train_10minutes['days']==28)]

# 把28号记录转化成小时级，因为28的十分钟级存在一定程度的随机性，而相对而言，小时级的稳定性更高

t28_in_hour = t28.groupby(['stationID', 'hours'])['inNums'].agg({'in28_mean_hour':'mean'}).reset_index()
t28_out_hour = t28.groupby(['stationID', 'hours'])['outNums'].agg({'out28_mean_hour':'mean'}).reset_index()
t28_in_out_hour = t28_in_hour.merge(t28_out_hour,on = ['stationID', 'hours'],how = 'left')

t28 = t28.merge(t28_in_out_hour,on = ['stationID', 'hours'],how = 'left')
t_28 = t28[['stationID', 'hours', 'minutes','inNums','outNums','in28_mean_hour','out28_mean_hour']]

### 3.5  预测： 29日的10min中预测结果 = 上周10min中的值 / 对应的小时均值 * 28日的小时均值

In [ ]:
sub29=pd.read_csv(path + 'testA_submit_2019-01-29.csv')
del sub29['inNums']
del sub29['outNums']

sub29['hours']  = pd.to_datetime(sub29['startTime'],format='%Y-%m-%d %H:%M:%S').dt.hour
sub29['minutes']= pd.to_datetime(sub29['startTime'],format='%Y-%m-%d %H:%M:%S').dt.minute

sub29=sub29.merge(t21_25_in_out_bili,on = ['stationID', 'hours', 'minutes'],how = 'left')
sub29=sub29.merge(t_28,on = ['stationID', 'hours', 'minutes'],how = 'left')
sub29=sub29.fillna(0)

* 此处乘以0.95是考虑到春运,人口会迅速下降，所以我们假设周二的总量小于周一的总量。


In [ ]:
sub29['in29'] = sub29['in_bili'] * sub29['in28_mean_hour']*0.95
sub29['out29']= sub29['out_bili']* sub29['out28_mean_hour']*0.95

submition=sub29[['stationID','startTime','endTime','in29','out29']]
submition=submition.rename(columns = {'in29':'inNums'})
submition=submition.rename(columns = {'out29':'outNums'})
submition.to_csv('baseline.csv',index=False)

### 4  小结

在这一小节,通过EDA发现本周一和上周一的分布类似,同时上周一到上周五的分布都是类似的,为了防止极值带来的影响,我们对小时和分钟取均值,然后计算比例, 然后用28日的小时均值近似本周的小时均值,乘以上一周的比例,从而得到29日的预测结果。

最后乘以0.95是假设周二的总量小于周一的总量(考虑到春运,人口会迅速下降), 通过上面的操作我们可以得到A榜12.10左右的分数。大概是58名的成绩。